In [1]:
import numpy as np
import torch
import torch.nn as nn
import pandas as pd
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
import time
import pydde as d

In [30]:
#Parameters
samplenum = 300
epochs = 100
hiddenlayers = [90]
input_size = 3
output_size = 3
learning_rate = 0.001
time_length = 60; #seconds

In [31]:
# Generate simulation
dyn = d.PyDyn('test2.sim', time_length)
state_init = dyn.compute(dyn.p_init)
f = dyn.f(state_init, dyn.p_init)
df = dyn.df_dp(state_init, dyn.p_init)
dy = dyn.dy_dp(state_init, dyn.p_init)

In [32]:
#Sample targets only variables in z direction
y_target = np.zeros((samplenum,3))
y_target[:,2] = np.random.rand(samplenum)
#x[:,0] = np.random.rand(samplenum)
y_target[:,1] = 2
y_target= torch.tensor(y_target)

## Building the custon Simulation Activation Function

In [33]:
class Simulate(torch.autograd.Function):
    
    @staticmethod
    def forward(ctx, input):
        #print(f'input: {input.shape}')
        p = input.clone().numpy().transpose()
        state = dyn.compute(p)
        y_pred = torch.tensor(state.y[-3:])
        #print(f'y_pred: {y_pred.dtype}')
        
        ctx.save_for_backward(input)
        
        return y_pred
    
    @staticmethod
    def backward(ctx, grad_output):
        #print(grad_output.shape)
        input, = ctx.saved_tensors
        p = input.clone().numpy().transpose()
        state= dyn.compute(p)
        dy_dp = dyn.dy_dp(state, p)
        dy_dp = dy_dp[-3:, :]
        #print(f'shape of dy/dp: {dy_dp.shape}')
        #print(f'shape of grad_output: {grad_output.shape}')
        grad_output = grad_output.unsqueeze(0)
        
        grad_input = torch.tensor(dy_dp).t().mm(grad_output.t()).t()
        return grad_input

Simulate = Simulate.apply

## Building the Model

In [34]:
class ActiveLearn(nn.Module):

    def __init__(self, n_in, out_sz):
        super(ActiveLearn, self).__init__()

        self.L_in = nn.Linear(n_in, hiddenlayers[0])
        self.H1 = nn.Linear(hiddenlayers[0], 3*time_length)
        #self.H1 = nn.Linear(hiddenlayers[0], hiddenlayers[1])
        #self.H2 = nn.Linear(hiddenlayers[1], 3*time_length)
        self.P = nn.Linear(3*time_length, 3*time_length)
        self.Relu = nn.ReLU(inplace=True)
    
    def forward(self, input):
        x = self.L_in(input)
        x = self.Relu(x)
        x = self.H1(x)
        x = self.Relu(x)
        #x = self.H2(x)
        #x = self.Relu(x)
        x = self.P(x)
        x = self.Relu(x)
        #x, p = Simulate(x)
        return x


model = ActiveLearn(input_size, output_size)

criterion = nn.MSELoss()  # RMSE = np.sqrt(MSE)
optimizer = torch.optim.Adam(model.parameters(), lr = learning_rate)

y_target = y_target.float()

## Train the model

In [35]:
torch.autograd.set_detect_anomaly(True)

start_time = time.time()

losses = []
y_preds= np.zeros((samplenum, 3))
p_preds= np.zeros((samplenum, 3*time_length))

#y_pred = torch.tensor(y_pred)
for i in range(epochs):
    for s in range(samplenum):
        y_truth = y_target[s, :]
        #print(y_truth.shape)
        #y_truth = y_truth.unsqueeze(0)
        p_pred = model(y_truth)
        y_pred = Simulate(p_pred)
        #print(y_pred.shape)
        y_preds[s, :] = y_pred.detach()
        p_preds[s, :] = p_pred.detach()
        #loss = torch.sqrt(criterion(y_pred.float(), y_truth)) # RMSE
        loss = criterion(y_pred.float(), y_truth) # MSE
        #loss =sum((y_pred.float()-y_truth)**2) 
        losses.append(loss)
        optimizer.zero_grad()
        #Back Prop
        loss.backward()
        optimizer.step()
    print(f'epoch: {i:3}/{epochs}  loss: {loss.item():10.8f}')
    i+=1

print(f'epoch: {i:3} loss: {loss.item():10.8f}') # print the last line
print(f'\nDuration: {(time.time() - start_time)/60:.3f} min') # print the time elapsed

epoch:   0/100  loss: 0.36655095
epoch:   1/100  loss: 0.67137098
epoch:   2/100  loss: 0.76946491
epoch:   3/100  loss: 0.29436484
epoch:   4/100  loss: 0.27473098
epoch:   5/100  loss: 0.24678572
epoch:   6/100  loss: 0.25862655
epoch:   7/100  loss: 0.24847789
epoch:   8/100  loss: 0.21911789
epoch:   9/100  loss: 0.22918035
epoch:  10/100  loss: 0.26954097
epoch:  11/100  loss: 0.21446466
epoch:  12/100  loss: 0.20759344
epoch:  13/100  loss: 0.16880524
epoch:  14/100  loss: 0.14307599
epoch:  15/100  loss: 0.12750436
epoch:  16/100  loss: 0.10503265
epoch:  17/100  loss: 0.08631773
epoch:  18/100  loss: 0.07082889
epoch:  19/100  loss: 0.05932863
epoch:  20/100  loss: 0.04957286
epoch:  21/100  loss: 0.04240880
epoch:  22/100  loss: 0.03471810
epoch:  23/100  loss: 0.02982222
epoch:  24/100  loss: 0.02583964
epoch:  25/100  loss: 0.03282196
epoch:  26/100  loss: 0.02410091
epoch:  27/100  loss: 0.02275795
epoch:  28/100  loss: 0.02210878
epoch:  29/100  loss: 0.03367528
epoch:  30

In [8]:
#Save Model

if len(losses) == epochs*(samplenum):
    torch.save(model.state_dict(), 'Trained_Model_300420_300s_100e_onlyZpos.pt')
    print('Model saved')
else:
    print('Model has not been trained. Consider loading a trained model instead.')

Model saved


## Test forward propagation

In [ ]:
p= np.random.rand(180)
p= torch.tensor(p)
y_pred = Simulate(p)
y_pred = y_pred.clone().numpy()

yTraj_test = sim.compute(p_pred)

print(y_pred)
print(yTraj_test[-3:])
print(np.sum(yTraj_test[-3:]-y_pred))

## Torch Script Conversion and Saving

In [36]:
input_example = torch.tensor(y_target[0,:])
traced_script_module = torch.jit.trace(model, input_example)

# Test the torch script
test_input= torch.tensor([0, 2, 0.5])
output_example = traced_script_module(test_input)
print(output_example[-5:])

# Save serialized model
traced_script_module.save("CPP_example_model_latest.pt")


tensor([1.9574, 0.0000, 0.0000, 2.7701, 0.0000], grad_fn=<SliceBackward>)
